# 자연어처리

RNN 을 활용한 텍스트 분류 (Text Classification)

## 확인

1. GPU 옵션 켜져 있는지 확인할 것!!! (수정 - 노트설정 - 하드웨어설정 (GPU))

## 순서

1. **import**: 필요한 모듈 import
2. **전처리**: 학습에 필요한 데이터 전처리를 수행합니다.
3. **모델링(model)**: 모델을 정의합니다.
4. **컴파일(compile)**: 모델을 생성합니다.
5. **학습 (fit)**: 모델을 학습시킵니다.

## 문제

-------------------------------
**자연어 처리**<br/><br/>

주어진 `spam.csv` file을 활용하여 스팸 메일 분류기를 만듭니다.

-----------------------------------


## 필요한 모듈 import

In [ ]:
import json
import tensorflow as tf
import numpy as np
import urllib
import pandas as pd

pd.set_option('display.max_colwidth', None)

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

## 필요한 데이터 셋 다운로드

In [ ]:
url = 'https://www.dropbox.com/s/iwm9kucrx5i6h2e/spam.csv?dl=1'
urllib.request.urlretrieve(url, 'spam.csv')

('spam.csv', <http.client.HTTPMessage at 0x7fb1eaec7050>)

## 데이터 로드

In [ ]:
df = pd.read_csv('spam.csv', encoding='latin1')
df = df[['v1', 'v2']]
df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"
...,...,...
5567,spam,"This is the 2nd time we have tried 2 contact u. U have won the å£750 Pound prize. 2 claim is easy, call 087187272008 NOW1! Only 10p per minute. BT-national-rate."
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other suggestions?"
5570,ham,The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free


## 전처리: 데이터셋 구성(sentences, labels)

다음과 같이 `v1` 컬럼의 `ham`은 0으로 `spam`은 1로 변환합니다.

In [ ]:
df['v1'] = df['v1'].apply(lambda x: 1 if x == 'spam' else 0)

In [ ]:
df['v1'].value_counts()

0    4825
1     747
Name: v1, dtype: int64

## train / valid 데이터셋 분리

In [ ]:
from sklearn.model_selection import train_test_split

SEED = 123

데이터셋 분할 옵션

- test_size=0.2
- stratify 지정
- random_state=SEED 지정

In [ ]:
train_sentence, valid_sentence, train_label, valid_label = train_test_split(df['v2'], df['v1'], 
                                                                            stratify=df['v1'],
                                                                            test_size=0.2, 
                                                                            random_state=SEED
                                                                           )

In [ ]:
train_sentence.shape, valid_sentence.shape

((4457,), (1115,))

In [ ]:
train_sentence[:5]

4128                                                                                                                                  Me i'm not workin. Once i get job...
134     Want 2 get laid tonight? Want real Dogging locations sent direct 2 ur mob? Join the UK's largest Dogging Network bt Txting GRAVEL to 69888! Nt. ec2a. 31p.msg@150p
2360                                             Had the money issue weigh me down but thanks to you, I can breathe easier now. I.ll make sure you dont regret it. Thanks.
3545                          SO IS TH GOWER MATE WHICH IS WHERE I AM!?! HOW R U MAN? ALL IS GOOD IN WALES ILL B BACK åÔMORROW. C U THIS WK? WHO WAS THE MSG 4? åÐ RANDOM!
3253                                                                                                                                I can make lasagna for you... vodka...
Name: v2, dtype: object

In [ ]:
train_label.shape, valid_label.shape

((4457,), (1115,))

In [ ]:
train_label[:5]

4128    0
134     1
2360    0
3545    0
3253    0
Name: v1, dtype: int64

## Tokenizer 정의

단어의 토큰화를 진행합니다.

* `num_words`: 단어 max 사이즈를 지정합니다. 가장 **빈도수가 높은** 단어부터 저장합니다.
* `oov_token`: 단어 토큰에 없는 단어를 어떻게 표기할 것인지 지정해줍니다.

In [ ]:
vocab_size = 800
oov_tok = "<OOV>"

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')

## Tokenizer로 학습시킬 문장에 대한 토큰화 진행

`fit_on_texts`로 학습할 문장에 대하여 **토큰화**를 진행합니다.

In [ ]:
tokenizer.fit_on_texts(train_sentence)

토큰화된 **단어 사전의 갯수**

In [ ]:
len(tokenizer.word_index)

7907

## 문장(sentences)을 Sequence로 변환

`texts_to_sequences`: 문장을 숫자로 **치환** 합니다. Train Set, Valid Set 모두 별도로 적용해주어야 합니다.

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_sentence)
validation_sequences = tokenizer.texts_to_sequences(valid_sentence)

## 시퀀스의 길이를 맞춰주기

3가지 옵션을 입력해 줍니다.

* `maxlen`: 최대 문장 길이를 정의합니다. 최대 문장길이보다 길면, 잘라냅니다.
* `truncating`: 문장의 길이가 `maxlen`보다 길 때 앞을 자를지 뒤를 자를지 정의합니다.
* `padding`: 문장의 길이가 `maxlen`보다 짧을 때 채워줄 값을 앞을 채울지, 뒤를 채울지 정의합니다.

In [ ]:
# 한 문장의 최대 단어 숫자
max_length = 120

# 잘라낼 문장의 위치
trunc_type='post'

# 채워줄 문장의 위치
padding_type='post'

In [ ]:
train_padded = pad_sequences(train_sequences, maxlen=max_length, truncating=trunc_type, padding=padding_type)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

## 모델 정의 (Sequential)

In [ ]:
embedding_dim = 16

In [ ]:
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Bidirectional(LSTM(32, return_sequences=True)),
    Bidirectional(LSTM(32)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

요약 확인

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           12800     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 120, 64)           12544     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 64)                24832     
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 52,801
Trainable params: 52,801
Non-trainable params: 0
__________________________________________________

## 컴파일 (compile)

1. `optimizer`는 가장 최적화가 잘되는 알고리즘인 'adam'을 사용합니다.
2. `loss`는 이진 분류이기 때문에 `binary_crossentropy`를 사용합니다.

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

## ModelCheckpoint: 체크포인트 생성

`val_loss` 기준으로 epoch 마다 최적의 모델을 저장하기 위하여, ModelCheckpoint를 만듭니다.
* `checkpoint_path`는 모델이 저장될 파일 명을 설정합니다.
* `ModelCheckpoint`을 선언하고, 적절한 옵션 값을 지정합니다.

In [ ]:
checkpoint_path = 'my_checkpoint.ckpt'
checkpoint = ModelCheckpoint(checkpoint_path, 
                             save_weights_only=True, 
                             save_best_only=True, 
                             monitor='val_loss',
                             verbose=1)

## 학습 (fit)

In [ ]:
model.fit(train_padded, train_label, 
          validation_data=(validation_padded, valid_label),
          callbacks=[checkpoint],
          epochs=10)

Epoch 1/10
140/140 [==============================] - 3s 19ms/step - loss: 0.0119 - acc: 0.9966 - val_loss: 0.0749 - val_acc: 0.9857

Epoch 00001: val_loss did not improve from 0.05349
Epoch 2/10
140/140 [==============================] - 3s 19ms/step - loss: 0.0072 - acc: 0.9984 - val_loss: 0.0895 - val_acc: 0.9839

Epoch 00002: val_loss did not improve from 0.05349
Epoch 3/10
140/140 [==============================] - 3s 19ms/step - loss: 0.0120 - acc: 0.9973 - val_loss: 0.0775 - val_acc: 0.9803

Epoch 00003: val_loss did not improve from 0.05349
Epoch 4/10
140/140 [==============================] - 3s 19ms/step - loss: 0.0077 - acc: 0.9982 - val_loss: 0.1106 - val_acc: 0.9785

Epoch 00004: val_loss did not improve from 0.05349
Epoch 5/10
140/140 [==============================] - 3s 19ms/step - loss: 0.0145 - acc: 0.9955 - val_loss: 0.0727 - val_acc: 0.9839

Epoch 00005: val_loss did not improve from 0.05349
Epoch 6/10
140/140 [==============================] - 3s 19ms/step - loss: 

## 학습 완료 후 Load Weights (ModelCheckpoint)

학습이 완료된 후에는 반드시 `load_weights`를 해주어야 합니다.

그렇지 않으면, 열심히 ModelCheckpoint를 만든 의미가 없습니다.

In [ ]:
model.load_weights(checkpoint_path)

## 모델 평가 (Evaluate)

`model.evaluate()`

In [ ]:
model.evaluate(validation_padded, validation_labels)

35/35 [==============================] - 0s 8ms/step - loss: 0.0535 - acc: 0.9892


[0.053488053381443024, 0.9892376661300659]